# Importamos bibliotecas

In [1]:
import pandas as pd
from PIL import Image
import numpy as np
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import pandas as pd
from PIL import Image
from skimage.feature import hog
import numpy as np
from joblib import load

# Procesado de imagenes a 16x16 para aligerar tiempos de entrenamiento, se convierte a escala de grises para poder extraer las Hog features, preferiblemente sería tenerlas en 64x64

In [2]:
#Procesado de img (a 16x16 para agilizar el entrenamiento)
def load_and_process_image(image_path):
    image = Image.open(image_path)
    image = image.resize((16, 16)) # Cambia el tamaño de la imagen a 16x16.
    image = image.convert('L') # Convertir a escala de grises ya que HOG se calcula sobre imágenes en escala de grises
    return np.array(image)

# Funcixtraccion de Hog features

In [3]:
#hog
def extract_features(images):
    hog_features = []
    for image in images:
        fd = hog(image, orientations=8, pixels_per_cell=(16,16), cells_per_block=(1, 1))
        hog_features.append(fd)
    return np.array(hog_features)


# Lectura de datos desde CSV de train, test y validation

In [4]:
#Csv
print("Cargando datos de los CSV...")
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')
val_data = pd.read_csv('val_data.csv')


Cargando datos de los CSV...


# Procesamos y obtenemos etiquetas de los 3 csv anteriormente leídos

In [5]:
#Carga+procesado+obtencion de etiquetas
print("procesando imágenes...")
X_train = np.array([load_and_process_image(img_path) for img_path in train_data['path']])
y_train = train_data['label']
X_test = np.array([load_and_process_image(img_path) for img_path in test_data['path']])
y_test = test_data['label']
X_val = np.array([load_and_process_image(img_path) for img_path in val_data['path']])
y_val = val_data['label']
print("imágenes procesadas con éxito.")


procesando imágenes...
imágenes procesadas con éxito.


# Extraccion de las hog features

In [6]:
# características HOG
print("Extrayendo HOG features...")
X_train_hog = extract_features(X_train)
X_test_hog = extract_features(X_test)
X_val_hog = extract_features(X_val)
print("HOG extraídas con éxito.")

Extrayendo HOG features...
HOG extraídas con éxito.


# Instancia de SVC, con la configuración por defecto utilizamos un RBF y parametros de regularización de 1.0

In [7]:
#clasificador SVM
clf = svm.SVC()

# Comienza el entrenamiento del modelo

print("Entrenando el modelo SVM...")
clf.fit(X_train_hog, y_train)
print("Al fin, entrenado.")

# Se realizan predicciones usando el conjunto de test y validation

In [9]:
# predicciones
print("Realizando predicciones en test_data...")
y_pred_test = clf.predict(X_test_hog)
print("Realizando predicciones en val_data...")
y_pred_val = clf.predict(X_val_hog)


Realizando predicciones en test_data...
Realizando predicciones en val_data...


# Rendimiento del modelo en los conjuntos de prueba y validación. La exactitud es del 71.7% en el conjunto de prueba y del 71.4% en el conjunto de validación. La precisión es la proporción de verdaderos positivos (TP) entre la suma de los verdaderos positivos y los falsos positivos (FP). La recuperación, también conocida como sensibilidad o tasa de verdaderos positivos, es la proporción de verdaderos positivos (TP) entre la suma de los verdaderos positivos y los falsos negativos (FN). La puntuación F1 es la media armónica de la precisión y la recuperación, proporcionando un equilibrio entre ambas medidas. El modelo presenta un sesgo hacia la clase 0, lo cual es normal al trabajar con un cojunto de datos desequilibrado.

In [10]:
# Evaluacion
print("Evaluando modelo...")
print('\nAccuracy en el test: ',accuracy_score(y_test, y_pred_test))
print('\n',classification_report(y_test, y_pred_test))
print('\nAccuracy en validación: ',accuracy_score(y_val, y_pred_val))
print('\n',classification_report(y_val, y_pred_val))


Evaluando modelo...

Accuracy en el test:  0.7179129933459848

               precision    recall  f1-score   support

           0       0.72      1.00      0.84     29886
           1       0.00      0.00      0.00     11743

    accuracy                           0.72     41629
   macro avg       0.36      0.50      0.42     41629
weighted avg       0.52      0.72      0.60     41629


Accuracy en validación:  0.7143097360013452

               precision    recall  f1-score   support

           0       0.71      1.00      0.83     29736
           1       0.00      0.00      0.00     11893

    accuracy                           0.71     41629
   macro avg       0.36      0.50      0.42     41629
weighted avg       0.51      0.71      0.60     41629



C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metr

# Save del modelo

In [11]:
# Guardar el modelo
from joblib import dump
dump(clf, 'svm_model.joblib') 
print("Modelo guardado con éxito.")

Modelo guardado con éxito.


# Predicciones del modelo, usaremos test y validation ya que n se han usado durante el entrenamiento

In [5]:
# Carga de datos (uso validacion por no usarse en el entrenamiento)
train_data = pd.read_csv('val_data.csv')

# Extraemos una imagen aleatoria de los conjuntos de datos para realizar la predicción

In [6]:
# Sacar una imagen aleatoria
random_row = train_data.sample()
image_path = random_row['path'].values[0]
true_label = random_row['label'].values[0]

# Función para la carga y procesado de la imagen anteriormente extraida, una vez más se procesa a 16x16

In [8]:
# Carga+procesado
def load_and_process_image(image_path):
    image = Image.open(image_path)
    image = image.resize((16, 16))
    image = image.convert('L')
    return np.array(image)

image = load_and_process_image(image_path)


# Función para extraer las Hog

In [9]:
#HOG
def extract_features(image):
    fd = hog(image, orientations=8, pixels_per_cell=(16,16), cells_per_block=(1, 1))
    return fd

hog_features = extract_features(image)

# Se carga el modelo guardado previamente y con el se realizan las predicciones, dando en este caso una prediccion de etiqueta 0 y siendo la verdadera etiqueta un 0

In [10]:
# Carga de modelo
model = load('svm_model.joblib')

#Predicción
prediction = model.predict([hog_features])

print("La predicción para la imagen es:", prediction)
print("La etiqueta verdadera de la imagen es:", true_label)

La predicción para la imagen es: [0]
La etiqueta verdadera de la imagen es: 0
